# Prepare

In [1]:
!pip install /workspace/RMSearch1-3/
# or
# pip install -r requirements.txt

Processing /workspace/RMSearch1-3
  Preparing metadata (setup.py) ... done
  Created wheel for rmsearch: filename=rmsearch-0.1.0-py3-none-any.whl size=7743 sha256=6d14c0bf08be8ecb214849d1f4ace406abd5405dd066d23540f214b4b4bd4bf5
  Stored in directory: /root/.cache/pip/wheels/9d/13/b2/fabbc0de47e01fe6408c2f07683041d9b31bf3ea04516d2a0d
Successfully built rmsearch
  Attempting uninstall: rmsearch
    Found existing installation: rmsearch 0.1.0
    Uninstalling rmsearch-0.1.0:
      Successfully uninstalled rmsearch-0.1.0

[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip


In [ ]:
# Run the following commands in terminal
'''
cd /workspace
pip install "huggingface_hub[hf_transfer]"
pip install hf_transfer
HF_HUB_ENABLE_HF_TRANSFER=1 huggingface-cli download Ray2333/GRM-Llama3.2-3B-rewardmodel-ft --local-dir ./llama3b-rm/
'''

# Search Keys from Queries

In [3]:
from rmsearch import Search
search = Search(model_name = "/workspace/llama3b-rm",
        tensor_parallel_size = 1,
        pipeline_parallel_size = 1,)

ImportError: cannot import name 'TypeIs' from 'typing_extensions' (/usr/local/lib/python3.11/dist-packages/typing_extensions.py)

In [ ]:
import time

queries = ["How to make LLM?", "What's the capital of Japan?"] * 5
keys = ["LLM is Large Language Model which can be made ..."*7, "Japanese capital is ..."*7] * 5
output = await search(queries, keys)


print(output)

In [3]:
output

[{'query': 'How to make LLM?',
  'query_id': 0,
  'keys': [{'key_id': 8,
    'key': 'LLM is Large Language Model which can be made ...'},
   {'key_id': 6, 'key': 'LLM is Large Language Model which can be made ...'},
   {'key_id': 0, 'key': 'LLM is Large Language Model which can be made ...'},
   {'key_id': 4, 'key': 'LLM is Large Language Model which can be made ...'},
   {'key_id': 2, 'key': 'LLM is Large Language Model which can be made ...'}]},
 {'query': "What's the capital of Japan?",
  'query_id': 1,
  'keys': [{'key_id': 5, 'key': 'Japanese capital is ...'},
   {'key_id': 9, 'key': 'Japanese capital is ...'},
   {'key_id': 7, 'key': 'Japanese capital is ...'},
   {'key_id': 1, 'key': 'Japanese capital is ...'},
   {'key_id': 3, 'key': 'Japanese capital is ...'}]},
 {'query': 'How to make LLM?',
  'query_id': 2,
  'keys': [{'key_id': 8,
    'key': 'LLM is Large Language Model which can be made ...'},
   {'key_id': 6, 'key': 'LLM is Large Language Model which can be made ...'},
  

# Train Example 1

In [2]:
exp_dir = "/workspace/exp12"
model_save_dir = f"{exp_dir}/model1"
model_name = "/workspace/llama3b-rm"
data_save_path = "/workspace/bigcodebench3.json"
advice_save_path = "/workspace/code-log5-corr1.json"
data_dict_save_path = f"{exp_dir}/data_dict.json"
dataset_list_save_path = f"{exp_dir}/dataset_list.json"
#num_advice_per_batch = 13  # total number of advice including both chosen and rejected per 1 batch
avoid_topk = 10 # avoid topk similar advices to each advice list being included in rejected advice

import json, os
if not os.path.exists(exp_dir):
    os.makedirs(exp_dir)
    

In [3]:
# Reference
# https://medium.com/towards-generative-ai/reward-model-training-2209d1befb5f

if not os.path.exists(data_dict_save_path):
    
    with open(data_save_path) as f:
        data_dict = json.load(f)
    
    task_id=data_dict["task_id"]
    complete_prompt=data_dict["complete_prompt"]
    instruct_prompt=data_dict["instruct_prompt"]
    canonical_solution=data_dict["canonical_solution"]
    code_prompt=data_dict["code_prompt"]
    test=data_dict["test"]
    doc_struct=data_dict["doc_struct"]
    num_problems = len(task_id)
    
    with open(advice_save_path) as f:
        log = json.load(f)
    
    data_dict = {"problem_id":[], "problem":[], "advice":[], "advice_id":[], "output":[], "correct":[], "log_ids":[], "node_state":[]}
    
    def search_log(log_dict, log_ids):  # node_id: str
        global data_dict
        node_id = log_ids[-1]
    
        if type(log_dict[node_id]) != dict:
            return None
            
        if "children" in log_dict[node_id]:
            for next_node_id in log_dict[node_id]["children"]:
                problem_id_str = log_ids[0]
                problem_id = int(problem_id_str)
                problem = instruct_prompt[problem_id]
                advice_list = log_dict[node_id]["children"][next_node_id]["instruction_list"]
    
                if "corrects" in log_dict[node_id]["children"][next_node_id]:  # when quiting the inference after finihing creating advice
                    all_correct = True
                    all_incorrect = True
                    child_num_correct = 0
                    child_num_problem = 0
                    for advice_id, advice in enumerate(advice_list):
                        correct = log_dict[node_id]["children"][next_node_id]["corrects"][str(advice_id)]
                        child_num_problem += 1
                        if correct:
                            all_incorrect = False
                            child_num_correct += 1
                        else:
                            all_correct = False
    
                    if child_num_problem == 0: continue
    
                    for advice_id, advice in enumerate(advice_list):
                        correct = log_dict[node_id]["children"][next_node_id]["corrects"][str(advice_id)]
                        #output = log_dict[node_id]["children"][next_node_id]["outputs"][str(advice_id)]
                        output = log_dict[node_id]["outputs"]["0"]
                        data_dict["problem_id"].append(problem_id)
                        data_dict["problem"].append(problem)
                        data_dict["output"].append(output)
                        data_dict["advice"].append(advice)
                        data_dict["advice_id"].append(advice_id)
                        data_dict["correct"].append(correct)
                        data_dict["log_ids"].append(log_ids+[next_node_id])
                        data_dict["node_state"].append({"num_problem":len(advice_list), "child_num_correct":child_num_correct, "all_correct":all_correct, "all_incorrect":all_incorrect})
    
                    if all_incorrect:
                        search_log(log_dict[node_id]["children"], log_ids + [next_node_id])
    
    problem_ids_with_advice = []
    for problem_id_str in log:
        if type(log[problem_id_str])==dict:
            if "children" in log[problem_id_str]:
                problem_ids_with_advice.append(int(problem_id_str))
                search_log(log, [problem_id_str])
    
    with open(f"{exp_dir}/data_dict.json", "w") as f: json.dump(data_dict, f)
    with open(f"{exp_dir}/problem_ids_with_advice.json", "w") as f: json.dump(problem_ids_with_advice, f)

else:
    with open(f"{exp_dir}/data_dict.json") as f: data_dict = json.load(f)
    with open(f"{exp_dir}/problem_ids_with_advice.json") as f: problem_ids_with_advice = json.load(f)



In [4]:
import torch

if not os.path.exists(f"{exp_dir}/advice_similarities.pt"):
    # Get similarities between advice so that avoid rejected advice being important for the problems
    from sentence_transformers import SentenceTransformer
    from sentence_transformers.util import cos_sim
    from sentence_transformers.quantization import quantize_embeddings
    
    # 1. Specify preffered dimensions
    dimensions = 512
    
    # 2. load model
    model = SentenceTransformer("mixedbread-ai/mxbai-embed-large-v1", truncate_dim=dimensions).to("cuda")
    advices = data_dict["advice"]
    # The prompt used for query retrieval tasks:
    # query_prompt = 'Represent this sentence for searching relevant passages: '
    
    # 2. Encode
    query_embedding = model.encode(advices, prompt_name="query")
    # Equivalent Alternatives:
    # query_embedding = model.encode(query_prompt + query)
    # query_embedding = model.encode(query, prompt=query_prompt)
    
    docs_embeddings = model.encode(advices)
    
    # Optional: Quantize the embeddings
    binary_query_embedding = quantize_embeddings(query_embedding, precision="ubinary")
    binary_query_embedding = torch.tensor(binary_query_embedding).to("cuda")
    binary_docs_embeddings = quantize_embeddings(docs_embeddings, precision="ubinary")
    binary_docs_embeddings = torch.tensor(binary_docs_embeddings).to("cuda")
    
    advice_similarities = cos_sim(query_embedding, docs_embeddings)
    torch.save(advice_similarities, f"{exp_dir}/advice_similarities.pt")
    
else:
    advice_similarities = torch.load(f"{exp_dir}/advice_similarities.pt")


/tmp/ipykernel_1399/4218478872.py:36: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  advice_similarities = torch.load(f"{exp_dir}/advice_similarities.pt")


In [5]:
import json
import pandas as pd

if not os.path.exists(dataset_list_save_path):
    df = pd.DataFrame(data_dict)
    df['id'] = range(len(df))
    dataset_list = []
    
    for problem_id in problem_ids_with_advice:
        #problem = instruct_prompt[problem_id]
        chosen_df = df[df['problem_id'] == problem_id]
        problem = chosen_df['problem'].values[0]
        output = chosen_df['output'].values[0]
        chosen_ids = chosen_df["id"].tolist()
        all_avoid_ids = []
        for chosen_id in chosen_ids:
            _, avoid_ids = torch.topk(advice_similarities[chosen_id], k=avoid_topk)
            avoid_ids = avoid_ids.tolist()
            all_avoid_ids += avoid_ids
        
        rejected_df = df.drop(all_avoid_ids)[df['problem_id'] != problem_id]
        num_chosen = len(chosen_df)
        num_rejected = num_chosen
        if not chosen_df.empty and not rejected_df.empty:
            '''
            if len_chosen_df<4:
                num_chosen = len_chosen_df
                num_rejected = num_advice_per_batch - len_chosen_df
            else: # For the case there are too many chosen advices
                num_chosen = 3
                num_rejected = num_advice_per_batch - num_chosen
            '''
            chosen_rows = chosen_df.sample(n=num_chosen)
            rejected_rows = rejected_df.sample(n=num_rejected)
            chosen_advice_ids = chosen_rows["id"].tolist()
            rejected_advice_ids = rejected_rows["id"].tolist()
            #chosen_reject_similarities = advice_similarities[chosen_ids][:, rejected_ids]
            for i in range(num_chosen):
                chosen_advice = chosen_rows['advice'].values[i]
                rejected_advice = rejected_rows['advice'].values[i]
                chosen_advice_id = chosen_advice_ids[i]
                rejected_advice_id = rejected_advice_ids[i]
                
                dataset_list.append({
                    "query":[{'role': 'user', 'content':f"Give me an advice to the problem and answer below;\n\nProblem:{problem}\n\nAnswer:{output}"}],
                    "chosen_key":[{'role': 'assistant', 'content': f"{chosen_advice}"}],
                    "rejected_key":[{'role': 'assistant', 'content': f"{rejected_advice}"}],
                    "problem_id":problem_id,
                    "problem":problem,
                    "output":output,
                    "chosen_advice":chosen_advice,
                    "rejected_advice":rejected_advice,
                    "chosen_advice_id":chosen_advice_id,
                    "rejected_advice_id":rejected_advice_id,
                })
        else:
            #print(f"problem {problem_id} doesn't have any advice")
            continue
    
    #if num_gpu*batch_size_per_device != 1:
    #    num_trash = len(dataset_list)%(num_gpu*batch_size_per_device)
    #    dataset_list = dataset_list[:-num_trash]
    
    with open(dataset_list_save_path, "w") as f:
        json.dump(dataset_list, f)
        
else:
    with open(dataset_list_save_path) as f:
        dataset_list = json.load(f)

#dataset1 = Dataset.from_list(dataset_list)
#dataset1.to_pandas()

In [6]:
from rmsearch import RMTrainer

model_name = "/workspace/llama3b-rm"
num_gpus = 1

rmtrainer = RMTrainer(model_name = model_name, num_gpus = num_gpus)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
dataset_save_path = f"{exp_dir}/dataset"
train_ids_save_path = f"{exp_dir}/train_ids.json"
test_ids_save_path = f"{exp_dir}/test_ids.json"
test_size = 48

formatted_dataset = rmtrainer.prepare_dataset(dataset_list, dataset_save_path, test_size, train_ids_save_path, test_ids_save_path)

In [ ]:
from trl import RewardConfig
from peft import LoraConfig, TaskType

batch_size_per_device = 4
eval_batch_size_per_device = 4

training_args = RewardConfig(
    output_dir=model_save_dir,
    per_device_train_batch_size=batch_size_per_device,
    per_device_eval_batch_size=eval_batch_size_per_device,
    #evaluation_strategy="steps",
    eval_steps=20,
    eval_on_start=True,
    save_steps=20,
    logging_steps=1,
    num_train_epochs = 3,
    report_to=None,
    remove_unused_columns=False,
)

peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    inference_mode=False,
    target_modules=["k_proj","q_proj","o_proj", "v_proj","down_proj","gate_proj","up_proj",],
    layers_to_transform=[25,26,27],
    r=16,
    lora_alpha=16,
    lora_dropout=0.1,
)

rmtrainer.train(formatted_dataset, training_args = training_args, peft_config = peft_config)

No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃ chosen_text                                   ┃ rejected_text                                ┃ logits           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ <|begin_of_text|><|start_header_id|>system<|… │ <|begin_of_text|><|start_header_id|>system<… │ [0.4907, 0.5093] │
│                                               │                                              │                  │
│ Cutting Knowledge Date: December 2023         │ Cutting Knowledge Date: December 2023        │                  │
│ Today Date: 07 May 2025                       │ Today Date: 07 May 2025                      │                  │
│                                               │                                              │                  │
│ <|eot_id|><|start_header_id|>user<|end_heade… │ <|eot_id|><|start_header_id|>user<|end_head… │                  │
│                                               │                                              │                  │
│ Give me an advice to the problem and answer   │ Give me an advice to the problem and answer  │                  │
│ below;                                        │ below;                                       │                  │
│                                               │                                              │                  │
│ Problem:Train a logistic regression model on  │ Problem:Train a logistic regression model on │                  │
│ one feature and evaluate its performance      │ one feature and evaluate its performance     │                  │
│ using a confusion matrix plot. The function   │ using a confusion matrix plot. The function  │                  │
│ takes a feature and a target series, splits   │ takes a feature and a target series, splits  │                  │
│ them into training and testing sets, trains   │ them into training and testing sets, trains  │                  │
│ the logistic regression model, predicts the   │ the logistic regression model, predicts the  │                  │
│ target for the test set, and plots the        │ target for the test set, and plots the       │                  │
│ confusion matrix.                             │ confusion matrix.                            │                  │
│ The function should output with:              │ The function should output with:             │                  │
│     (np.ndarray, plt.Axes): A tuple           │     (np.ndarray, plt.Axes): A tuple          │                  │
│ containing the confusion matrix and the       │ containing the confusion matrix and the      │                  │
│ matplotlib Axes object of the confusion       │ matplotlib Axes object of the confusion      │                  │
│ matrix plot.                                  │ matrix plot.                                 │                  │
│ You should write self-contained code starting │ You should write self-contained code         │                  │
│ with:                                         │ starting with:                               │                  │
│ ```                                           │ ```                                          │                  │
│ import pandas as pd                           │ import pandas as pd                          │                  │
│ from sklearn.model_selection import           │ from sklearn.model_selection import          │                  │
│ train_test_split                              │ train_test_split                             │                  │
│ from sklearn.linear_model import              │ from sklearn.linear_model import             │                  │
│ LogisticRegression                            │ LogisticRegression                           │                  │
│ from sklearn.metrics import confusion_matrix  │ from s

Step,Training Loss,Validation Loss,Accuracy
0,No log,0.339076,0.812500
1,0.382000,No Log,No Log
2,0.230500,No Log,No Log
3,0.418800,No Log,No Log
4,0.627400,No Log,No Log
5,0.531700,No Log,No Log
6,0.303600,No Log,No Log
7,0.060400,No Log,No Log
8,0.028500,No Log,No Log
9,0.242800,No Log,No Log


# Train Example 2